In [55]:
import pandas as pd
import requests
import urllib
from datetime import date


# define API constants
BASE_URL = 'https://api.prop-odds.com'
API_KEY = '' # enter your API key here (from https://prop-odds.com/profile)

# define NBA seasons
season_2022_2023 = pd.date_range(date(2023, 3, 30), date(2023, 6, 12)) # data starts from March 3, 2023
season_2023_2024 = pd.date_range(date(2023, 10, 24), date.today())
nba_seasons = [season_2022_2023, season_2023_2024]

# create dataframe to hold date, home_team, away_team, sportsbook, home_team_odds, away_team_odds, winner
games_basic_info = pd.DataFrame(columns=['date', 'home_team', 'away_team', 'prop_odds_game_id', 'sportsbook', 'home_team_odds', 'away_team_odds', 'winner'])
games_with_odds = pd.DataFrame(columns=['date', 'home_team', 'away_team', 'prop_odds_game_id', 'sportsbook', 'home_team_odds', 'away_team_odds', 'winner'])

def get_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

    print('Request failed with status:', response.status_code)
    print(response.reason)
    return {}

In [56]:
def get_nba_games_on_date(date):
    """Returns on object containing all nba games on the given date
        Parameters:
            date (datetime.date): date to get games for
        Returns:
            games_on_date: dict object containing all nba games on the given date
    """
    query_params = {
        'date': date.strftime('%Y-%m-%d'),
        'tz': 'America/New_York',
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/games/nba?' + params
    return get_request(url)

In [57]:
def get_nba_games_in_date_range(date_range):
    """Inserts basic information about each nba game in the date range into games_df
        Parameters:
            date_range (pd.date_range): date range to get games for
        Returns:
            None
    """
    for date in date_range:
        games_on_date = get_nba_games_on_date(date)
        # games_on_date = {'games': [{'home_team': 'Bucks', 'away_team': 'Lakers'}, {'home_team': 'Warriors', 'away_team': 'Nets'}]}
        if games_on_date:
            for game in games_on_date['games']:
                # append to end of games_df
                games_basic_info.loc[len(games_basic_info)] = [date, game['home_team'], game['away_team'], game['game_id'], '', '', '', '']

In [58]:
def fill_basic_game_info():
    """Calls get_nba_games_in_date_range for each season in nba_seasons"""
    for season in nba_seasons:
        get_nba_games_in_date_range(season)

In [59]:
def get_game_odds(game_id, game_date, market='moneyline'):
    """Returns an object containing the odds provided by the sportsbooks for the given game and market
        Parameters: 
            game_id (str): prop odds game id
            market (str): market to get odds for (moneyline, spread, total)
        Returns:
            odds: dict object containing the odds for the given game and market
    """
    query_params = {
        'api_key': API_KEY,
        'end_datetime': game_date.strftime('%Y-%m-%d') + 'T12:00:00',
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/odds/' + game_id + '/' + market + '?' + params
    return get_request(url)

In [60]:
def identify_home_away(outcomes, home_team, away_team):
    """Given a list of outcome objects, identifies the home and away outcome objects
        Parameters:
            outcomes (list): list of outcome objects
            home_team (str): home team name
            away_team (str): away team name
        Returns:
            home_outcome: outcome object for the home team
            away_outcome: outcome object for the away team
    """
    home_outcome = None
    away_outcome = None
    home_team_name = home_team.split(' ')[-1]
    away_team_name = away_team.split(' ')[-1]
    for outcome in outcomes:
        if home_team_name in outcome['name']:
            home_outcome = outcome
        elif away_team_name in outcome['name']:
            away_outcome = outcome
    return home_outcome, away_outcome

In [61]:
def add_odds_for_single_game(game_id, game_date):
    """Adds odds for a single game to games_full_data for all available sportsbooks
        Parameters:
            game_id (str): prop odds game id
        Returns:
            None
    """
    odds = get_game_odds(game_id, game_date)
    if odds:
        for sportsbook in odds['sportsbooks']:
            sportsbook_name = sportsbook['bookie_key']
            game = games_basic_info[games_basic_info['prop_odds_game_id'] == game_id].copy()
            game['sportsbook'] = sportsbook_name
            home_outcome, away_outcome = identify_home_away(sportsbook['market']['outcomes'], game['home_team'].values[0], game['away_team'].values[0])
            game['home_team_odds'] = home_outcome['odds']
            game['away_team_odds'] = away_outcome['odds']
            games_with_odds.loc[len(games_with_odds)] = game.values[0]
        

In [62]:
def fill_odds():
    """Calls add_odds_for_single_game for each game in games_df"""
    for game_id, game_date in zip(games_basic_info['prop_odds_game_id'], games_basic_info['date']):
        add_odds_for_single_game(game_id, game_date)

In [69]:
# uncomment out two lines below to get games and odds, will consume many api calls and take ~2.5 minutes
# fill_basic_game_info()
# fill_odds()
games_with_odds.to_csv('games_with_odds.csv', index=False)

In [64]:
# load csv into dataframe
games_full_data = pd.read_csv('games_with_odds.csv')

In [65]:
# use balldontlie api to get winner for each game
start_date, end_date = games_with_odds['date'].min(), games_with_odds['date'].max()
per_page = 100

def get_page_of_results(page):
    """Returns a page of results from the balldontlie api
        Parameters:
            page (int): page number to get results for
        Returns:
            results: dict object containing a page of results from the balldontlie api
    """
    query_params = {
        'start_date': start_date,
        'end_date': end_date,
        'per_page': per_page,
        'page': page
    }
    params = urllib.parse.urlencode(query_params)
    url = 'https://www.balldontlie.io/api/v1/games?' + params
    return get_request(url)

In [66]:
def get_all_results():
    """Returns a list of all results from the balldontlie api
        Parameters:
            None
        Returns:
            results: list object containing all results from the balldontlie api
    """
    results = []
    current_page = 1
    while True:
        page_results = get_page_of_results(current_page)
        total_pages = page_results['meta']['total_pages']
        current_page += 1
        results += page_results['data']
        if current_page == total_pages:
            break
    return results

In [67]:
def fill_game_results(results):
    """Adds the winner to games_full_data for each game in results
        Parameters:
            results (list): list of game results from the balldontlie api
        Returns:
            None
    """
    for result in results:
        date = result['date'][:10]
        home_team = result['home_team']['full_name']
        away_team = result['visitor_team']['full_name']
        if result['home_team_score'] and result['visitor_team_score']:
            winner = home_team if result['home_team_score'] > result['visitor_team_score'] else away_team
            games_full_data.loc[(games_with_odds['date'] == date) & (games_full_data['home_team'] == home_team) & (games_full_data['away_team'] == away_team), 'winner'] = winner
        

In [68]:
results = get_all_results()
fill_game_results(results)
games_full_data.to_csv('games_full_data.csv', index=False)